In [1]:
# Import functions and libraries
import numpy as np
import matplotlib.pyplot as plt
import queue as Queue
import time
import sys

from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from numpy import ones
from scipy import signal
from scipy import integrate
import threading

from numpy import mean
from numpy import power
from numpy.fft import fft
from numpy.fft import fftshift
from numpy.fft import ifft
from numpy.fft import ifftshift
import bitarray
from  scipy.io.wavfile import read as wavread
import newax25 as ax25

import multiprocessing

from math import gcd
import sounddevice as sd
import RPi.GPIO as GPIO
from functools import reduce
from numpy import ones,zeros, pi, cos, exp, sign



In [2]:
# function to compute least common multipler
def lcm(numbers):
    return reduce(lambda x, y: (x*y)//gcd(x,y), numbers, 1)

In [3]:
sd.query_devices()

  0 bcm2835 ALSA: - (hw:0,0), ALSA (0 in, 2 out)
  1 bcm2835 ALSA: IEC958/HDMI (hw:0,1), ALSA (0 in, 2 out)
  2 Fe-Pi Audio: - (hw:1,0), ALSA (2 in, 2 out)
  3 sysdefault, ALSA (0 in, 128 out)
  4 dmix, ALSA (0 in, 2 out)
* 5 default, ALSA (2 in, 2 out)

In [4]:
builtin_idx = 0
usb_idx = 2
sd.default.samplerate=48000
sd.default.channels = 1

In [24]:
import numpy.ctypeslib as npct
from ctypes import c_int
from ctypes import c_float

array_1d_int = npct.ndpointer(dtype=np.int, ndim=1, flags='CONTIGUOUS')

libcd = npct.load_library("./libpll", ".")
libcd.pll.restype = c_int
libcd.pll.argtypes= [array_1d_int, c_int, array_1d_int,array_1d_int,  array_1d_int,array_1d_int, c_int, c_float]



class TNCaprs:
    
    def __init__(self, fs = 48000.0, Abuffer = 512, Nchunks=10,baud=1200,center_f=1700):
        
        #  Implementation of an afsk1200 TNC. 
        #
        #  The TNC processes a `Abuffer` long buffers, till `Nchunks` number of buffers are collected into a large one.
        #  This is because python is able to more efficiently process larger buffers than smaller ones.
        #  Then, the resulting large buffer is demodulated, sampled and packets extracted.
        #
        # Inputs:
        #    fs  - sampling rate
        #   TBW  -  TBW of the demodulator filters
        #   Abuffer - Input audio buffers from Pyaudio
        #   Nchunks - Number of audio buffers to collect before processing
        #   plla    - agressivness parameter of the PLL
        
        
        ## compute sizes based on inputs
        self.baud = baud  
        self.center_f = center_f
        self.TBW = 2.0   # TBW for the demod filters
        self.N = (int(fs/self.baud*self.TBW)//2)*2+1   # length of the mark-space filters for demod
        self.fs = fs     # sampling rate   
        self.BW = 1200      # BW of filter based on TBW
        self.Abuffer = Abuffer             # size of audio buffer
        self.Nchunks = Nchunks             # number of audio buffers to collect
        self.Nbuffer = Abuffer*Nchunks+(self.N*3-3)         # length of the large buffer for processing
        self.space_f = self.center_f+500
        self.mark_f = self.center_f-500
        self.Ns = 1.0*fs/self.baud # samples per symbol
        ## state variables for the modulator
        self.prev_ph = 0  # previous phase to maintain continuous phase when recalling the function
                         
        ##  Generate Filters for the demodulator
        self.h_lp = signal.firwin(self.N,self.BW/self.fs*1.0,window='hanning')
        self.h_lpp = signal.firwin(self.N,self.BW*2*1.2/self.fs,window='hanning')
        #self.h_lpp = signal.firwin(self.N,self.BW*2*3/self.fs,window='hanning')
        self.h_space = self.h_lp*exp(1j*2*pi*(self.space_f)*r_[-self.N/2:self.N/2]/self.fs)
        self.h_mark = self.h_lp*exp(1j*2*pi*(self.mark_f)*r_[-self.N/2:self.N/2]/self.fs)
        self.h_bp = (signal.firwin(self.N,self.BW/self.fs*2.2,window='hanning'))*exp(1j*2*pi*self.center_f*r_[-self.N/2:self.N/2]/self.fs)
        #self.h_bp = (signal.firwin(self.N,self.BW/self.fs*5,window='hanning'))*exp(1j*2*pi*self.center_f*r_[-self.N/2:self.N/2]/self.fs)


        
        

        ## PLL state variables  -- so conntinuity between buffers is preserved
        self.dpll = np.round(2.0**32 / self.Ns).astype(np.int32)    # PLL step
        self.pll =  0                # PLL counter
        self.ppll = -self.dpll       # PLL counter previous value -- to detect overflow
        self.plla = 0.74             # PLL agressivness (small more agressive)
        

        ## state variable to NRZI2NRZ
        self.NRZIprevBit = bool(1)  
        
        ## State variables for findPackets
        self.state='search'   # state variable:  'search' or 'pkt'
        self.pktcounter = 0   # counts the length of a packet
        self.packet = bitarray.bitarray([0,1,1,1,1,1,1,0])   # current packet being collected
        self.bitpointer = 0   # poiter to advance the search beyond what was already searched in the previous buffer

        ## State variables for processBuffer
        self.buff = zeros(self.Nbuffer)   # large overlapp-save buffer
        self.chunk_count = 0              # chunk counter
        self.oldbits = bitarray.bitarray([0,0,0,0,0,0,0])    # bits from end of prev buffer to be copied to beginning of new
        self.Npackets = 0                 # packet counter
        
        
    
    
    def NRZ2NRZI(self,NRZ, prevBit = True):
        NRZI = NRZ.copy() 
        for n in range(0,len(NRZ)):
            if NRZ[n] :
                NRZI[n] = prevBit
            else:
                NRZI[n] = not(prevBit)
            prevBit = NRZI[n]
        return NRZI
    



    def NRZI2NRZ(self, NRZI):  
        NRZ = NRZI.copy() 
    
        for n in range(0,len(NRZI)):
            NRZ[n] = NRZI[n] == self.NRZIprevBit
            self.NRZIprevBit = NRZI[n]
    
        return NRZ
    
    def KISS2bits(self,KISS):
        # function that takes a KISS frame sent via TCP/IP and converts it to an APRSpacket bit stream.
        
        bits = bitarray.bitarray(endian="little")
        bits.frombytes(KISS)
        fcs = ax25.FCS()
        for bit in bits:
            fcs.update_bit(bit)
            
        bits.frombytes(fcs.digest())
        return bitarray.bitarray('01111110') + ax25.bit_stuff(bits) + bitarray.bitarray('01111110') 
     
    def bits2KISS(self,bits):
        # function that takes a bitstream of an APRS-packet, removes flags and FCS and unstuffs the bits
        bitsu = ax25.bit_unstuff(bits[8:-8])
        return  bitsu[:-16].tobytes() 
    
    
    def modulate(self,bits):
    # the function will take a bitarray of bits and will output an AFSK1200 modulated signal of them, sampled at fs Hz
    #  Inputs:
    #         bits  - bitarray of bits
    #         fs    - sampling rate
    # Outputs:
    #         sig    -  returns afsk1200 modulated signal 
        # For you to complete
        fss = lcm((self.baud,self.fs))
        deci = fss//self.fs
        Nb = fss//self.baud
        print("Nb:",Nb)
        nb = len(bits)
        NRZ = ones((nb,Nb))
        for n in range(0,nb):
            if bits[n]:
                NRZ[n,:]=-NRZ[n,:]
    
        freq = self.center_f + 500*NRZ.ravel()
        #freq = 4000 + 2000*NRZ.ravel()
        ph = 2.0*pi*integrate.cumtrapz(freq)/fss
        sig = cos(ph[::deci])
        return sig
        

    
    def modulatPacket(self, callsign, digi, dest, info, preflags=80, postflags=80 ):
        
        # given callsign, digipath, dest, info, number of pre-flags and post-flags the function contructs
        # an appropriate aprs packet, then converts them to NRZI and calls `modulate` to afsk 1200 modulate the packet. 
        
        packet = ax25.UI(destination=dest,source=callsign, info=info, digipeaters=digi.split(b','),)
        prefix = bitarray.bitarray(np.tile([0,1,1,1,1,1,1,0],(preflags,)).tolist())
        suffix = bitarray.bitarray(np.tile([0,1,1,1,1,1,1,0],(postflags,)).tolist())
        sig = self.modulate(self.NRZ2NRZI(prefix + packet.unparse()+suffix))

        return sig
    
    

    def demod(self, buff):
        #Demodulates a buffer and returns valid NRZ
        # Similar to afsk1200_demod,  for you to complete

        
        buff = np.convolve(buff.copy(),self.h_bp,'valid')
        mark = abs(np.convolve(buff,self.h_mark,mode='valid'))
        space = abs(np.convolve(buff,self.h_space,mode='valid'))
        NRZ = mark-space
        NRZ = np.convolve(NRZ,self.h_lpp,'valid')
        return NRZ



    def FastPLL(self,NRZa):
        recbits = np.zeros(len(NRZa)//(self.fs//self.baud)*2,dtype=np.int32)
        pll = np.zeros(1,dtype = np.int32)
        pll[0] = self.pll
        ppll = np.zeros(1,dtype = np.int32)
        ppll[0] = self.ppll
        
        #print("pll = ",pll,"   ppll=",ppll)
        
        
        NRZb = (NRZa > 0).astype(np.int32)
        tot = libcd.pll(NRZb,len(NRZb),recbits,recbits,pll,ppll,self.dpll,self.plla)
        
        self.ppll = ppll.copy()
        self.pll = pll.copy()
        
        #print("post: pll = ",pll,"   ppll=",ppll)
        
        return bitarray.bitarray(recbits[:tot].tolist())
    
    def PLL(self, NRZa):
       #print("running PLL")
        idx = zeros(len(NRZa)//int(self.Ns)*2)   # allocate space to save indexes        
        c = 0
        
        for n in range(1,len(NRZa)):
            if (self.pll < 0) and (self.ppll >0):
                idx[c] = n
                c = c+1
        
            if (NRZa[n] >= 0) !=  (NRZa[n-1] >=0):
                self.pll = np.int32(self.pll*self.plla)
            
        
            self.ppll = self.pll
            self.pll = np.int32(self.pll+ self.dpll)
    
        return idx[:c].astype(np.int32) 
    
   

    def findPackets(self,bits):
        # function take a bitarray and looks for AX.25 packets in it. 
        # It implements a 2-state machine of searching for flag or collecting packets
        flg = bitarray.bitarray([0,1,1,1,1,1,1,0])
        packets = []
        n = self.bitpointer
        
        # Loop over bits
        while (n < len(bits)-7) :
            # default state is searching for packets
            if self.state is 'search':
                # look for 1111110, because can't be sure if the first zero is decoded
                # well if the packet is not padded.
                if bits[n:n+7] == flg[1:]:
                    # flag detected, so switch state to collecting bits in a packet
                    # start by copying the flag to the packet
                    # start counter to count the number of bits in the packet
                    self.state = 'pkt'
                    self.packet=flg.copy()
                    self.pktcounter = 8
                    # Advance to the end of the flag
                    n = n + 7
                else:
                    # flag was not found, advance by 1
                    n = n + 1            
        
            # state is to collect packet data. 
            elif self.state is 'pkt':
                # Check if we reached a flag by comparing with 0111111
                # 6 times ones is not allowed in a packet, hence it must be a flag (if there's no error)
                if bits[n:n+7] == flg[:7]:
                    # Flag detected, check if packet is longer than some minimum
                    if self.pktcounter > 200:
                        #print('packet found!')
                        # End of packet reached! append packet to list and switch to searching state
                        # We don't advance pointer since this our packet might have been
                        # flase detection and this flag could be the beginning of a real packet
                        self.state = 'search'
                        self.packet.extend(flg)
                        packets.append(self.packet.copy())
                    else:
                        # packet is too short! false alarm. Keep searching 
                        # We don't advance pointer since this this flag could be the beginning of a real packet
                        self.state = 'search'
                # No flag, so collect the bit and add to the packet
                else:
                    # check if packet is too long... if so, must be false alarm
                    if self.pktcounter < 2680:
                        # Not a false alarm, collect the bit and advance pointer        
                        self.packet.append(bits[n])
                        self.pktcounter = self.pktcounter + 1
                        n = n + 1
                    else:  #runaway packet
                        #runaway packet, switch state to searching, and advance pointer
                        self.state = 'search'
                        n = n + 1
        
        self.bitpointer = n-(len(bits)-7) 
        return packets

    
    # function to generate a checksum for validating packets
    def genfcs(self,bits):
        # Generates a checksum from packet bits
        fcs = ax25.FCS()
        for bit in bits:
            fcs.update_bit(bit)
    
        digest = bitarray.bitarray(endian="little")
        digest.frombytes(fcs.digest())

        return digest




    # function to parse packet bits to information
    def decodeAX25(self,bits, deepsearch=False):
        ax = ax25.AX25()
        ax.info = "bad packet"
    
    
        bitsu = ax25.bit_unstuff(bits[8:-8])
    
        
        #foundPacket = False
        #if (self.genfcs(bitsu[:-16]).tobytes() == bitsu[-16:].tobytes()):
        #        foundPacket = True
        #elif deepsearch: 
        #    tbits = bits[8:-8]
        #    for n in range(0,len(tbits)):
        #        tbits[n] = not tbits[n]
        #        if (self.genfcs(bitsu[:-16]).tobytes() == bitsu[-16:].tobytes()):
        #            foundPacket = True
        #            print("Success deep search")
        #            break
        #        tbits[n] = not tbits[n]
        # 
        #if foundPacket == False:
        #    return ax
        
        if (self.genfcs(bitsu[:-16]).tobytes() == bitsu[-16:].tobytes()) == False:
            #print("failed fcs")
            return ax
                  
    
        bytes = bitsu.tobytes()
        ax.destination = ax.callsign_decode(bitsu[:56]).decode('ascii')
        source = ax.callsign_decode(bitsu[56:112]).decode('ascii')
    
        if source[-1].isdigit() and source[-1]!="0":
            ax.source = "".join((source[:-1],'-',source[-1]))
        else:
            ax.source = source[:-1]
    
        digilen=0    
    
        if bytes[14]=='\x03' and bytes[15]=='\xf0':
            digilen = 0
        else:
            for n in range(14,len(bytes)-1):
                if bytes[n] & 1:
                    digilen = (n-14)+1
                    break

        #    if digilen > 56:
        #        return ax
        ax.digipeaters =  ax.callsign_decode(bitsu[112:112+digilen*8]).decode('ascii')
        ax.info = bitsu[112+digilen*8+16:-16].tobytes()
    
    
        return ax

    def processBuffer(self, buff_in):
        
        # function processes an audio buffer. It collect several small into a large one
        # Then it demodulates and finds packets.
        #
        # The function operates as overlapp and save
        # The function returns packets when they become available. Otherwise, returns empty list
        
        N = self.N
        NN = (N*3 -3 )
        
        
        Nchunks = self.Nchunks
        Abuffer = self.Abuffer
        fs = self.fs
        Ns = self.Ns
        
        validPackets=[]
        packets=[]
        NRZI=[]
        idx = []
        bits = []
        
        # Fill in buffer at the right place
        self.buff[NN+self.chunk_count*Abuffer:NN+(self.chunk_count+1)*Abuffer] = buff_in.copy()
        self.chunk_count = self.chunk_count + 1
        
        
        # number of chunk reached -- process large buffer
        if self.chunk_count == Nchunks:
            # Demodulate to get NRZI
            NRZI = self.demod(self.buff)
            # compute sampling points, using PLL
            #idx = self.PLL(NRZI)
            # Sample and make a decision based on threshold
            #bits = bitarray.bitarray((NRZI[idx]>0).tolist())
            
            bits = self.FastPLL(NRZI)
            #bits = self.PLL(NRZI)
            # In case that buffer is too small raise an error -- must have at least 7 bits worth
            if len(bits) < 7:
                raise ValueError('number of bits too small for buffer')
            
            # concatenate end of previous buffer to current one
            bits = self.oldbits + self.NRZI2NRZ(bits)
            
            # store end of bit buffer to next buffer
            self.oldbits = bits[-7:].copy()
            
            # look for packets
            packets = self.findPackets(bits)
            
            # Copy end of sample buffer to the beginning of the next (overlapp and save)
            self.buff[:NN] = self.buff[-NN:].copy()
            
            # reset chunk counter
            self.chunk_count = 0
            
            # checksum test for all detected packets
            for n in range(0,len(packets)):
                if len(packets[n]) > 200: 
                    try:
                        ax = self.decodeAX25(packets[n])
                    except:
                        ax = ax25.AX25()
                        ax.info = "bad packet"
                    if ax.info != 'bad packet':
                        validPackets.append(packets[n])
                        
            
        return validPackets

In [25]:
GPIO.cleanup()
GPIO.setmode(GPIO.BOARD)
PTT = 16
GPIO.setup(PTT, GPIO.OUT, initial = 0)




In [33]:
callsign = "KN6AHX"
digi = b'WIDE1-1,WIDE2-1'
dest = "APCAL"
fname = b"calBlue.tiff"
f = open(fname,"rb")
contents = f.read()
print(len(contents))
fs = 48000
modem = TNCaprs(fs = fs ,Abuffer = 512,Nchunks = 10,baud=2400,center_f = 2400)
Qout = Queue.Queue()
info = bitarray.bitarray(256)
print(int(len(contents)/256)+1)
total_lengh = 0
for i in range(int(len(contents)/256)+1):
    info=contents[i*256:(i+1)*256]
    sig = modem.modulatPacket(callsign, digi, dest, info, preflags=80, postflags=80 )
    total_lengh+=len(sig)
    Qout.put(sig)
print(total_lengh)

8746
35
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
Nb: 20
2505205


In [34]:
while not Qout.empty():
    packet = Qout.get()
    print(i)
    i=i+1
    #print(packet)
    if packet is None:
        break
    GPIO.output(PTT, GPIO.HIGH) # Key radio
    time.sleep(0.1) #give radio time to start

    sd.play(packet*0.5,samplerate=48000,device=usb_idx,  blocking=True)  
    GPIO.output(PTT, GPIO.LOW)    
  
print("DONE1")


34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
DONE1


In [29]:
npack=0
print("decoding packets")
starttime = time.time()
count=0
# For you to complete
sig=[]
while not Qout.empty():
    p = Qout.get()
    #print(sig)
    if sig is None:
        break
    sig=sig+(p.tolist())
npack=0
sigp=np.zeros((int(len(sig)/512)+1)*512)
sig = np.array(sig)
sigp[0:len(sig)]=sig
print(sig.shape)
for i in range(int(len(sig)/512)+1):
    packets  = modem.processBuffer(sig[i*512:512*(i+1)])
    for pkt in packets: 
        npack = npack + 1
        ax = modem.decodeAX25(pkt)
        infostr = "%(n) 2i) | DEST: %(dest)s | SRC: %(src)s | DIGI: %(digi)s | %(info)s |" % {
                        'n': npack,
                        'dest': ax.destination,
                        'src': ax.source,
                        'digi': ax.digipeaters,
                        'info': ax.info.strip()
                    }
        print(infostr)
print(count)

decoding packets
(5010445,)
 1) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'MM\x00*\x00\x00\x19\xa0\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x03\x00B\x00\x00B\x00\x00B\x16\x0fA\x0e\x00B\x00\x00D\x00\x00G+(Cu_8\xa8\x86/\xc5\x9d(\xd1\xa7 \xc2\x9b&wd;\x00\x00F\x00\x00B\n\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x01\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x03\x00B\x00\x00B\x0c\x04B\x19\x11B\x00\x00F\x00\x00DOE=\xa9\x8a-\xe2\xb2\x19\xfc\xc4\x08\xff\xc8\x00\xff\xc9\x00\xff\xc8\x00\xff\xcb\x00\xfb\xc8\t\x8fu2\x00\x00D\x14\rC\x01\x00B\x00\x00' |
 2) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x0

 11) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'B\x00\x00B\x04\x00B\x18\x10B\x00\x00E\x00\x00F(#@o\\6\x99z0\xa8\x81,\x92v450@XI<\xa4\x83/\xa9\x84,\x98z0B5?\x00\x00C\x00\x00E|i8\xfb\xc5\x0c\xfd\xc2\n\xfc\xc1\x0b\xff\xc8\x00\x94y4\x00\x00J\x91v5\xfd\xc7\x06\xff\xcb\x00\xac\x8c/\x00\x00E\n\x00B\x0c\x04B\x00\x00B\x19\x11B\x00\x00E\x8dt4\xff\xc8\x01\xfe\xc2\t\xfb\xc1\n\xfd\xc2\t\xfc\xc2\n\xfd\xc3\x0b\xff\xcc\x00\xac\x8b*\x00\x00F\x18\x10B\x00\x00B\x01\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x06\x00B\x15\rB\x00\x00F*"C\x9f\x800\xde\xb3\x19\xf8\xc4\x06\xff\xc9\x00\xff\xcb\x00\xff\xcb\x00\xd4\xac\x1c\xe3\xb5\x17\xff\xcd\x00\xff\xca\x00\xff\xd0\x00\x90w4\x00\x00F\x00\x06B\xcb\xa3$\xff\xca\x00\xf8\xbe\x0e\xff\xc7\x00\xe5\xb5\x16\x00\x13DH=?\xed\xbd\x13\xff\xc9\x00\xea\xb5\x17I<A\x00\x00D\x1a\x12A\x00\x00B\x02\x00B' |
 12) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'\x00\x00C\x11\x18B\xd3\xa9\x1d\xff\xca\x00\xfa\xbf\x0c\xfd\xc2\t\xfd\xc2\t\xfb\xc1\x0b\xff\xc7\x06\xee\x

 21) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'\xdf\xb3\x19\xfa\xc3\n\xfb\xc0\x0b\xff\xc7\x06\xf4\xc2\x0c\xbd\x98%mZ9\x0e\x0fF\x00\x00D\x0b\x00B\x00\x00B\x00\x00B\x00\x00B\n\x00C\x14\nC\x00\x00F0*A\xa0\x83/\xd7\xac\x1e\xf1\xbc\x0f\xfc\xc4\x00\xff\xca\x00\xff\xc8\x00\xf3\xbe\x0f\xd5\xa7\x1f\x93v2\x00\x06C\x00\x00J\x00\x00I\x00\x00IcT9\xc7\xa0$\xe1\xb0\x1a\xe1\xaf\x18\xd0\xa4!\x9b~/\x12\x1dD}i6\xde\xaf\x1a\xeb\xb8\x15\xec\xb9\x13\xe9\xb6\x16\xd9\xa9\x1c\xb9\x95)yd:\x00\x00G\x00\x00J\x00\x00H\x00\x00F\x00\x00E\x00\x00H\x00\x00J%*A\xcd\xa4!\xff\xc8\x00\xf8\xbf\x0e\xff\xc4\t\xff\xc8\x00\xff\xca\x00\xac\x8d,\x00\x00D\x00\x00B\x0e\x06B\x00\x00B\x01\x00B\x00\x00B\x04\x00B\x19\x11A\x00\x00D\x00\x00G\x19\x1dD_O=\x80d6\x82i6r]:WJ<3.B2&AbN;\x81g7\x88l4\x8aq5\x80h9\x8ft5\xae\x8b,\xb5' |
 22) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'\x90&\xae\x8c*\xa2\x82/\xa6\x850\xa5\x85-\xba\x95\'\xd2\xa8\x1e\xc9\xa2\x1f\xbb\x96%\xa2\x84+zf7hV=oY8v^7v^9}f9\x88o5\x99{2\xb7\x91,\xc5\x

 31) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'\x00n\x00\xe9\x00r\x00i\x00c\x00o\x04\x17\x040\x043\x040\x04;\x04L\x04=\x048\x049\x00 \x04?\x04@\x04>\x04D\x040\x049\x04;\x00 \x00R\x00G\x00B\x00P\x00r\x00o\x00f\x00i\x00l\x00 \x00g\x00\xe9\x00n\x00\xe9\x00r\x00i\x00q\x00u\x00e\x00 \x00R\x00V\x00B\x90\x1au(\x00 \x00R\x00G\x00B\x00 \x82r_ic\xcf\x8f\xf0\x00P\x00r\x00o\x00f\x00i\x00l\x00o\x00 \x00R\x00G\x00B\x00 \x00g\x00e\x00n\x00e\x00r\x00i\x00c\x00o\x00G\x00e\x00n\x00e\x00r\x00i\x00s\x00k\x00 \x00R\x00G\x00B\x00-\x00p\x00r\x00o\x00f\x00i\x00l\xc7|\xbc\x18\x00 \x00R\x00G\x00B\x00 \xd5\x04\xb8\\\xd3\x0c\xc7|\x00O\x00b\x00e\x00c\x00n\x00\xfd\x00 \x00R\x00G\x00B\x00 \x00p\x00r\x00o\x00f\x00i\x00l\x05\xe4\x05\xe8\x05\xd5' |
 32) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'\x05\xe4\x05\xd9\x05\xdc\x00 \x00R\x00G\x00B\x00 \x05\xdb\x05\xdc\x05\xdc\x05\xd9\x00A\x00l\x00l\x00g\x00e\x00m\x00e\x00i\x00n\x00e\x00s\x00 \x00R\x00G\x00B\x00-\x00P\x00r\x00o\x00f\x00i\x00l\x00\xc

ValueError: could not broadcast input array from shape (13) into shape (512)

In [11]:
def queueREPLAY_callback(indata,outdata, frames, time, status):
    if status:
        print(status)
    outdata[:] = indata
    Qin.put( indata.copy()[:,0] )  # Global queue

st_replay = sd.Stream( samplerate=48000,device=(usb_idx,builtin_idx),callback=queueREPLAY_callback)
Qin = Queue.Queue()  
print("Starting streams")
st_replay.start()
starttime = time.time()
print("decoding packets")
starttime = time.time()
fname1 = b"rec_calBlue.tiff"
f1 = open(fname1,"wb")
while not Qout.empty():
    packet = Qout.get()
    #print(packet)
    if packet is None:
        break
    sd.play(packet*0.5,samplerate=48000,device=(usb_idx),blocking=True)




Starting streams
decoding packets


In [12]:
# For you to complete
npack = 0
while not(Qin.empty()):
    packets  = modem.processBuffer(Qin.get()) 
    ax = modem.decodeAX25(packets)
    for pkt in packets: 
        npack = npack + 1
        ax = modem.decodeAX25(pkt)
        infostr = "%(n) 2i) | DEST: %(dest)s | SRC: %(src)s | DIGI: %(digi)s | %(info)s |" % {
                        'n': npack,
                        'dest': ax.destination,
                        'src': ax.source,
                        'digi': ax.digipeaters,
                        'info': ax.info.strip()
                    }
        print(infostr)
        f1.write(ax.info)


print(time.time() - starttime)

f1.close()
st_replay.close()


 1) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'MM\x00*\x00\x00\x19\xa0\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x03\x00B\x00\x00B\x00\x00B\x16\x0fA\x0e\x00B\x00\x00D\x00\x00G+(Cu_8\xa8\x86/\xc5\x9d(\xd1\xa7 \xc2\x9b&wd;\x00\x00F\x00\x00B\n\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x01\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x03\x00B\x00\x00B\x0c\x04B\x19\x11B\x00\x00F\x00\x00DOE=\xa9\x8a-\xe2\xb2\x19\xfc\xc4\x08\xff\xc8\x00\xff\xc9\x00\xff\xc8\x00\xff\xcb\x00\xfb\xc8\t\x8fu2\x00\x00D\x14\rC\x01\x00B\x00\x00' |
 2) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00

 11) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'\x00\x00C\x11\x18B\xd3\xa9\x1d\xff\xca\x00\xfa\xbf\x0c\xfd\xc2\t\xfd\xc2\t\xfb\xc1\x0b\xff\xc7\x06\xee\xbc\x13I>@\x00\x00F\x16\x0eA\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x04\x00B\x16\rC\x00\x00G]P:\xd9\xad\x1b\xff\xcc\x00\xff\xc9\x00\xfd\xc2\x0b\xfa\xc0\x0e\xfc\xc1\r\xfc\xc1\x0c\xff\xc7\x04\xff\xc6\x08\xf9\xc0\x0e\xff\xc4\x08\xf1\xc0\x10cR:\x00\x00HmY9\xf5\xc1\x0c\xff\xc5\x08\xfa\xbf\r\xff\xcb\x00\xa5\x850\x00\x00F\xd4\xa7\x1d\xff\xcb\x00\xff\xc7\x00\xa2\x81/\x00\x00F\x17\x0eB\x03\x00B\x00\x00B\x00\x00B\x00\x00Et`7\xf8\xc5\x05\xfe\xc3\n\xfb\xc1\n\xfd\xc2\t\xfd\xc2\t\xfa\xc0\x0c\xff\xcb\x00\xbc\x97(\x00\x00G\x0e\x00D\t\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x18\x10B\x00\x00EgZ<\xee\xbf\x14\xff\xca\x00\xfa\xc1\x0e\xf9\xbf\x0c\xfc\xc2\x0b\xff\xc8\x00\xff\xc8\x00\xfd\xc3\x08\xfb\xc0\x0c\xfc\xc1\n\xfa\xbf\r\xff\xc9\x00\xc6' |
 12) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'\x9e%\x00\x00E\x00\x00F\x

 21) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'\x90&\xae\x8c*\xa2\x82/\xa6\x850\xa5\x85-\xba\x95\'\xd2\xa8\x1e\xc9\xa2\x1f\xbb\x96%\xa2\x84+zf7hV=oY8v^7v^9}f9\x88o5\x99{2\xb7\x91,\xc5\x9e&\xe8\xb7\x18\xff\xc6\x08\xfc\xc2\x0b\xf9\xbf\r\xfa\xbe\x0f\xff\xd0\x00\xb7\x96(\x00\x00D\x00\x00B\x0e\x07B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x00\x00B\x15\x0fB"\x19A\x00\x00D\x00\x00J\x00\x00GA9@\x8ds4\xb6\x94*\xd8\xab!\xf0\xbc\x12\xf9\xc4\x07\xfd\xc4\x04\xff\xc4\x02\xff\xc7\x00\xff\xc8\x00\xff\xc4\x05\xfc\xc2\x0b\xfd\xc2\x0b\xff\xc7\x00\xff\xcb\x00\xff\xcc\x00\xff\xcb\x00\xff\xc5\x05\xfe\xc3\t\xfb\xc1\n\xf9\xc1\x0b\xfc\xc2\t\xfe\xc3\x00\xfe\xc5\x00\xfb\xc3\x04\xf9\xc1\t\xfa\xc2\x07\xfd\xc4\x07\xff\xc6\x00\xff\xc7\x00\xff\xc9\x00\xff\xca\x00\xff\xc5\x04\xfb\xc0\x0c\xfe\xc4\t\xff\xc9\x00\xff\xcc\x00\xc6\x9f$,(B\x00\x00D\x13\x0bB\x00\x00B\x00\x00B\x00\x00B\x01\x00B\x00\x00' |
 22) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'B\x07\x00B\x14\x0cC\x00\x00E\x00\x00D\x86m7\xd0\xa7

 31) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b'\x05\xe4\x05\xd9\x05\xdc\x00 \x00R\x00G\x00B\x00 \x05\xdb\x05\xdc\x05\xdc\x05\xd9\x00A\x00l\x00l\x00g\x00e\x00m\x00e\x00i\x00n\x00e\x00s\x00 \x00R\x00G\x00B\x00-\x00P\x00r\x00o\x00f\x00i\x00l\x00\xc1\x00l\x00t\x00a\x00l\x00\xe1\x00n\x00o\x00s\x00 \x00R\x00G\x00B\x00 \x00p\x00r\x00o\x00f\x00i\x00lfn\x90\x1a\x00 \x00R\x00G\x00B\x00 c\xcf\x8f\xf0e\x87N\xf6N\x00\x82,\x00 \x00R\x00G\x00B\x00 0\xd70\xed0\xd50\xa10\xa40\xeb\x00P\x00r\x00o\x00f\x00i\x00l\x00 \x00R\x00G\x00B\x00 \x00g\x00e\x00n\x00e\x00r\x00i\x00c\x03\x93\x03\xb5\x03\xbd\x03\xb9\x03\xba\x03\xcc\x00 \x03\xc0\x03\xc1\x03\xbf\x03\xc6\x03\xaf\x03\xbb\x00 \x00R\x00G\x00B\x00P\x00e\x00r\x00f\x00i\x00l\x00 \x00R\x00G\x00B\x00 \x00g\x00e\x00n\x00\xe9' |
 32) | DEST: APCAL 0 | SRC: KN6AHX | DIGI: WIDE1 1WIDE2 1 | b"\x00R\x00G\x00B\x00 \x06'\x06D\x069\x06'\x06E\x00G\x00e\x00n\x00e\x00r\x00i\x00c\x00 \x00R\x00G\x00B\x00 \x00P\x00r\x00o\x00f\x00i\x00l\x00e\x00G\x00e\x00n\x00e\x0